In [2]:
import flor
import plotly.express as px

In [3]:
df = flor.load_kvs()
df

,projid,vid,recrep,epoch,step,name,value
0,kaggle-nlp-disasters-rnn,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,a,1,0,avg_train_loss,0.69547
1,kaggle-nlp-disasters-rnn,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,a,1,1,avg_train_loss,0.697254
2,kaggle-nlp-disasters-rnn,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,a,1,0,average_valid_loss,0.684606
3,kaggle-nlp-disasters-rnn,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,a,1,1,average_valid_loss,0.683746
4,kaggle-nlp-disasters-rnn,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,a,2,0,avg_train_loss,0.685091
...,...,...,...,...,...,...,...
5660,kaggle-nlp-disasters-rnn,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,a,80,1,avg_train_loss,0.371584
5661,kaggle-nlp-disasters-rnn,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,a,80,0,average_valid_loss,0.53874
5662,kaggle-nlp-disasters-rnn,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,a,80,1,average_valid_loss,0.507311
5663,kaggle-nlp-disasters-rnn,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,a,80,0,hindsight-logged-val,"[15, 30, 45, 60, 75, 90, 105, 120, 135, 150, 1..."


In [4]:
avg_train_loss = df[df['name'] == 'avg_train_loss'][['vid', 'epoch', 'step', 'value']]
avg_train_loss

,vid,epoch,step,value
0,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,1,0,0.69547
1,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,1,1,0.697254
4,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,2,0,0.685091
5,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,2,1,0.69357
8,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,3,0,0.677779
...,...,...,...,...
5648,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,78,1,0.35462
5653,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,79,0,0.374228
5654,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,79,1,0.34743
5659,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,80,0,0.370882


In [7]:
avg_val_loss = df[df['name'] == 'average_valid_loss'][['vid', 'epoch', 'step', 'value']]
avg_val_loss

,vid,epoch,step,value
2,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,1,0,0.684606
3,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,1,1,0.683746
6,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,2,0,0.680935
7,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,2,1,0.680846
10,c42e5f0314fe034c99dfbb16ea05d851aadd8c76,3,0,0.678437
...,...,...,...,...
5650,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,78,1,0.520838
5655,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,79,0,0.567907
5656,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,79,1,0.519202
5661,e5c85b6799f23ec7ec239eb2ae621b075f8933cd,80,0,0.53874


In [10]:
fig = px.scatter_3d(avg_val_loss, x='epoch', y='step', z='value' color='vid')
fig.show()

In [21]:
df['name'].unique()

array(['avg_train_loss', 'average_valid_loss', 'best_acc', 'best_loss'],
      dtype=object)

In [8]:
best_loss = df[df['name'] == 'best_loss']
best_loss

,projid,vid,recrep,epoch,step,name,value
4600,kaggle-nlp-disasters-rnn,08cfd10afdb763a8756868fdf48c132ea800957f,b,0,0,best_loss,inf
4601,kaggle-nlp-disasters-rnn,08cfd10afdb763a8756868fdf48c132ea800957f,b,1,0,best_loss,inf
4602,kaggle-nlp-disasters-rnn,08cfd10afdb763a8756868fdf48c132ea800957f,b,2,0,best_loss,inf
4603,kaggle-nlp-disasters-rnn,08cfd10afdb763a8756868fdf48c132ea800957f,b,3,0,best_loss,inf
4604,kaggle-nlp-disasters-rnn,08cfd10afdb763a8756868fdf48c132ea800957f,b,4,0,best_loss,inf
...,...,...,...,...,...,...,...
5165,kaggle-nlp-disasters-rnn,c104f9e7f253f236b9b0ab94a53d70bc927cf6ac,b,75,0,best_loss,inf
5166,kaggle-nlp-disasters-rnn,c104f9e7f253f236b9b0ab94a53d70bc927cf6ac,b,76,0,best_loss,inf
5167,kaggle-nlp-disasters-rnn,c104f9e7f253f236b9b0ab94a53d70bc927cf6ac,b,77,0,best_loss,inf
5168,kaggle-nlp-disasters-rnn,c104f9e7f253f236b9b0ab94a53d70bc927cf6ac,b,78,0,best_loss,inf


In [9]:
best_loss_valid = best_loss[best_loss['value'] != float('inf')]

In [10]:
fig = px.line(best_loss_valid, x='epoch', y='value', color='vid')
fig.show()